# Enter your name and password and select the other options then hit interact. Scroll down to see the results, takes about 3 mins to get them.


## Ogranizing by serial_number makes it easier to see the calculations are correct. 
## NaN or inf just means it didn't have the data to calculate that, since some machines weren't online for some days.




In [ ]:
import os
import time
from dataclasses import dataclass, fields
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
# see if I even need this time import. if nothing breaks delete it
# import time
import operator as op #op.itemgetter() method should help with grabbing values from json dicts
import json
import warnings

import numpy as np
import pandas as pd
import requests
import dotenv
import pytz
from dotenv import load_dotenv
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets 
from tqdm.notebook import tqdm as progbar
import matplotlib as mpl

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199
load_dotenv(".env")

In [ ]:
@dataclass
class Equipment:
    type: str
    model: str
    serial: str
machine_types = ['Bulldozer', 'Bulldozer', 'Bulldozer', 'Bulldozer', 'Bulldozer', 'Bulldozer', 'Bulldozer', 'Bulldozer',
                'Bulldozer', 'Bulldozer', 'Bulldozer', 'Bulldozer', 'Bulldozer', 'Bulldozer', 'Bulldozer',
                'Hydraulic excavator', 'Hydraulic excavator', 'Hydraulic excavator','Hydraulic excavator']
models = ['D61PX-24', 'D61PX-24', 'D61PX-24', 'D71PX-24', 'D61PX-24', 'D61PX-24', 'D61PX-24', 'D61PX-24', 'D61PX-24', 'D61PX-24', 'D61PX-24', 'D61PX-24', 'D61PX-24', 'D61PX-24', 'D61PX-24', 'PC490LC-11', 'PC360LC-11', 'PC360LC-11', 'PC360LC-11']
serials = ['45059', '45052', '45029', '70598', '45429', '45352', 'B65883', '45817', 'B65938', '45755', '45830', 'B65936', 'B65878', 'B65937', '45845', 'A42522', 'A37995', 'A37993', 'A37998']



In [ ]:
EQUIPMENT_LIST = [Equipment(x,y,z) for x,y,z in zip(machine_types, models, serials)]

USERNAME = "aa"
PASSWORD = "aa"

API_KOMATSU_TOKEN_URL='https://komtraxisoapi.azure-api.net/provider/token'
API_KOMATSU_FLEET_URL='https://komtraxisoapi.azure-api.net/provider/v1/140370/Fleet'
API_KOMATSU_EQUIPMENT_URL = 'https://komtraxisoapi.azure-api.net/provider/v1/140370/Fleet/Equipment/MakeModelSerial'


MAKE_CODE = '0001'

In [ ]:
def litres_to_gallons(litres):
    gallons = float(litres) * 0.264172
    return round(gallons, 2)

In [ ]:
def create_fleet_data_url(page_number: int = 1) -> str:
    """Creates a url for accessing fleet data

    Args:
        page_number (int, optional): The page number 
        Defaults to 1.

    Returns:
        str: _description_
    """
    return f'{FLEET_URL}/{page_number}/'



def create_historical_url(equipment:Equipment, start_UTC:str, end_UTC:str) -> str:
    """Creates a url for a accessing single piece of equipment's historical data.
    This is the only way to access historical data for Komatsu equipment.

    Args:
        equipment (Equipment): An instance of the Equipment dataclass
        start_UTC (str): The start time in UTC
        end_UTC (str): The end time in UTC

    Returns:
        str: A url for requesting the historical data for a single piece of equipment
    """
    return f'{EQUIPMENT_URL}/{MAKE_CODE}/{equipment.model}/{equipment.serial}/{start_UTC}/{end_UTC}/'



def create_machine_info_url(equipment:Equipment, start_UTC:str, end_UTC:str, info_type:str="CumulativeNonproductiveIdleHours") -> str:
    """Creates a url for a accessing single piece of equipment's historical data.
    This is the only way to access historical data for Komatsu equipment.

    Args:
        equipment (Equipment): An instance of the Equipment dataclass
        start_UTC (str): The start time in UTC
        end_UTC (str): The end time in UTC

    Returns:
        str: A url for requesting the historical data for a single piece of equipment
    """
    return f'{EQUIPMENT_URL}/{MAKE_CODE}/{equipment.model}/{equipment.serial}/{info_type}/{start_UTC}/{end_UTC}/'

In [ ]:
def utc_str_to_tz_str(utc_datetime_string:str=None, from_format:str='%Y-%m-%dT%H:%M:%SZ',
                       tz="US/Central", to_format:str="%Y-%m-%d %H:%M:%S") -> str:

    if utc_datetime_string is None:
        utc_datetime_string = datetime.utcnow().strftime(from_format)
    
    #convert string to datetime object
    datetime_object = datetime.strptime(utc_datetime_string, from_format)
    #convert to proper utc format
    utcmoment = datetime_object.replace(tzinfo=ZoneInfo('UTC'))

    #convert to local datetime then return as a string
    local_datetime = utcmoment.astimezone(ZoneInfo(tz))
    return local_datetime.strftime(to_format)

In [ ]:
def request_komatsu_api_token(username: str, password:str):
    """Requests a token from Komatsu's API
    
    Args:
        username: URL encoded username string
        password: URL encoded password string
        
    Returns:
        str: an access token string
    """
    payload = "grant_type=password&username=" + username + "&password=" + password
    headers = {"content-type": "application/x-www-form-urlencoded"}

    response = requests.request("POST", TOKEN_URL, data=payload, headers=headers)
    try:
        return response.json()["access_token"]
    except:
        raise Exception(response.text)
    

def request_komatsu_data(token:str, url:str) -> json:
    """Requests data from Komatsu's API

    Args:
        token (str): An authentication token to access the API
        url (str): The request url

    Returns:
        json: A json object containing the requested data
    """

    payload = "="
    headers = {
    "Authorization": "Bearer " + token,
    "content-type": "application/x-www-form-urlencoded"
    }
    response = requests.request("GET", url, data=payload, headers=headers)
    return response.json()


def request_with_retry(request_url:str, username:str, password:str):
    """Makes a request using request_komatsu_data() 
    and if successful returns the data.
    Will try for an hour before giving up. 

    Args:
        request_url (str): the full url to make a request of the komatsu api
        username (str): url-encoded username in string form
        password (str): url_incoded password in string form

    Returns:
        dict: data from the api
    """
    token = request_komatsu_api_token(username, password)

    stop = 0
    attempt_num = 0
    current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("komatsu_request_log_file.txt", "a") as f:
        f.write(f"\n\nlog data date:{current_datetime}\nrequest url: {request_url}\n")
        while stop == 0:
            #exit after x attempts
            attempt_num += 1
            if attempt_num > 7200:
                f.write("Too many attempts: exiting function\n")
                print("Too many attempts: exiting function")
                stop = 1
                continue
            #request data
            data = request_komatsu_data(token, request_url)
            status_code = data.get("statusCode", None)
            if status_code is None:
                message = data.get('Message', None)
                if message == 'Authentication is not correct':
                    token = request_komatsu_api_token(username, password)
                    f.write("Authentication is not correct. Requested new token\n")
                    continue
            # if too many attempts status
            if status_code == 429:
                if attempt_num % 100 == 0:
                    f.write(data["message"] + "\n")
                time.sleep(0.5)
                continue
            # if got something else aside from too many attempts status
            # return the data
            else:
                f.close()
                return data
        f.write(f"Failed to get request info:\n")
        f.close()
    return None

In [ ]:
def get_komatsu_historical_datapoint(username:str, password:str, days_of_data:int, end_date:datetime, datapoint:str="CumulativeOperatingHours") -> list:
    """Gets the historical data for a datapoint and returns a list of it.

    Args:
        equipment_list (list): _description_
        days_of_data (_type_): _description_
        end_date (_type_): _description_
        datapoint (_type_): FuelUsedInThePreceding24Hours
                            CumulativeNonproductiveIdleHours
                            CumulativeOperatingHours

    Returns:
        str: The filename to make it easy access the files after the function runs
    """
    #end_date = datetime.strptime(end_date, "%Y-%m-%d")
    start_date = end_date - timedelta(days=days_of_data)
    end_date = end_date.strftime("%Y-%m-%d")
    start_date = start_date.strftime("%Y-%m-%d")
    username = requests.utils.quote(username, safe='')
    password = requests.utils.quote(password, safe='')
    
    
    filename = f"{datapoint}_{start_date}_to_{end_date}.json"
    ret_list = []
    eq_length = len(EQUIPMENT_LIST)
    bar_length = eq_length
    bar = progbar(range(0,bar_length), desc='data request progress')
    for x in EQUIPMENT_LIST:
        url = create_machine_info_url(x, start_date, str(end_date), datapoint)
        ret_list.append(request_with_retry(url, username, password))
        bar.update(1)
    ret_dict = {datapoint: ret_list}
    bar.update(1)
    return ret_dict

def create_dataframe(data, datapoint):

    data = data[datapoint]
    
    model_list = []
    serial_list = []
    datetime_list = []
    hour_list = []
    fuel_list = []
    
    # have to make the P non-capital because of how its named in the data
    if datapoint == "CumulativeNonProductiveIdleHours":
        datapoint = "CumulativeNonproductiveIdleHours"
    elif datapoint == "FuelUsedInThePreceding24Hours":
        datapoint = "FuelUsedLast24"
    for x in data:
        string_list = x["Links"][0]["href"].split("/")
        model = string_list[10]
        serial = string_list[11]

        for d in x[datapoint]:
            model_list.append(model)
            serial_list.append(serial)
            cst_time = utc_str_to_tz_str(utc_datetime_string=d["datetime"],
                                         from_format='%Y-%m-%dT%H:%M:%SZ',
                                         tz="US/Central",
                                         to_format="%Y-%m-%d")
            datetime_list.append(cst_time)
            if datapoint == "FuelUsedLast24":
                fuel_list.append(float(d["FuelConsumed"]))
            else:
                hour_list.append(float(d["Hour"]))
    
    if datapoint == "FuelUsedLast24":
        final_data = np.array([serial_list, fuel_list, datetime_list]).T
    else:
        final_data = np.array([serial_list, hour_list, datetime_list]).T
        
    if datapoint == "CumulativeOperatingHours":
        df = pd.DataFrame(final_data, columns=["serial_number", "cumulative_op_hours", "louisiana_datetime"], )
        df = df.infer_objects()
        df["cumulative_op_hours"] = df["cumulative_op_hours"].astype(float)
        df["louisiana_datetime"] = pd.to_datetime(df["louisiana_datetime"])
        df = df.sort_values(by=["serial_number", "louisiana_datetime"])
        df["op_hours"] = df.groupby("serial_number")["cumulative_op_hours"].diff()
    elif datapoint == "CumulativeNonproductiveIdleHours":
        df = pd.DataFrame(final_data, columns=["serial_number", "cumulative_idle_hours", "louisiana_datetime"], )
        df = df.infer_objects()
        df["cumulative_idle_hours"] = df["cumulative_idle_hours"].astype(float)
        df["louisiana_datetime"] = pd.to_datetime(df["louisiana_datetime"])
        df = df.sort_values(by=["serial_number", "louisiana_datetime"])
        df["idle_hours"] = df.groupby("serial_number")["cumulative_idle_hours"].diff()
    elif datapoint == "FuelUsedLast24":
        df = pd.DataFrame(final_data, columns=["serial_number", "fuel_used_last_24", "louisiana_datetime"])
        df["louisiana_datetime"] = pd.to_datetime(df["louisiana_datetime"])
        df["fuel_used_last_24"] = df["fuel_used_last_24"].apply(lambda x: litres_to_gallons(x))
        df = df.sort_values(by=["serial_number", "louisiana_datetime"])
    return df

In [ ]:

datapoints = ["CumulativeOperatingHours",'CumulativeNonProductiveIdleHours','FuelUsedInThePreceding24Hours']

def full_widget_funct_all_data(username, password, days_of_data, end_date, display_style):
    datapoints = ["CumulativeOperatingHours",'CumulativeNonProductiveIdleHours','FuelUsedInThePreceding24Hours']
    dfs = {}
    
    def make_style(styler):
        styler.format("{:.2f}")
        styler.highlight_between(subset="idle_hours_/_op_hours", left=0.30, right=1.0, props='font-weight:bold;color:#b30000')
        return styler


    
    #create individual dataframes
    for datapoint in datapoints:
        print(f'Requesting {datapoint}')
        data = get_komatsu_historical_datapoint(username, password, days_of_data, end_date, datapoint)
        dfs[datapoint] = create_dataframe(data, datapoint)
    
    op_df = dfs["CumulativeOperatingHours"]
    idle_df = dfs['CumulativeNonProductiveIdleHours']
    fuel_df = dfs['FuelUsedInThePreceding24Hours']
    
    #merge into 1 dataframe
    df_final = pd.merge(op_df, idle_df, how="outer", on=["serial_number", "louisiana_datetime"])
    df_final = pd.merge(df_final, fuel_df, how="outer", on=["serial_number", "louisiana_datetime"])
    df_final["idle_hours_/_op_hours"] = df_final["idle_hours"] / df_final["op_hours"]
    
    df_final = df_final[["louisiana_datetime","serial_number","idle_hours_/_op_hours", "op_hours", "idle_hours","fuel_used_last_24", "cumulative_op_hours", "cumulative_idle_hours"]].copy()
    
    if display_style == "by_date":
        df_final = df_final.pivot_table(index=["louisiana_datetime","serial_number"])[["idle_hours_/_op_hours", "op_hours", "idle_hours","fuel_used_last_24", "cumulative_op_hours", "cumulative_idle_hours"]]
    else:
        df_final = df_final.pivot_table(index=["serial_number", "louisiana_datetime"])[["idle_hours_/_op_hours", "op_hours", "idle_hours","fuel_used_last_24", "cumulative_op_hours", "cumulative_idle_hours"]]

    display(df_final.style.pipe(make_style))
    

In [ ]:
out = widgets.Output(layout={'border': '4px solid blue'})

int_slider_days = widgets.IntSlider(
    value=10,
    min=1,
    max=30,
    step=1,
    description='Days of Data:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

text_user = widgets.Text(
    value='a',
    placeholder='Type something',
    description='Username:',
    disabled=False
)

text_pass = widgets.Text(
    value= 'a',
    placeholder='Type something',
    description='Password:',
    disabled=False
)

date_picker = widgets.DatePicker(
    description='Ending Date',
    disabled=False
)

style_dropdown = widgets.Dropdown( options=[('By Date', "by_date"),
                                    ('By Serial Number', 'by_serial_number')],
                                      value="by_serial_number",
                                      description='Display Style',
                                      disabled=False)


wout = widgets.ToggleButtons( options=[('Off', False),('On',True)],
                          description='Load Chart?',
                          disabled=False,
                          button_style='success')

def display_out():
    "Display widget for dataframe creation"
    o = widgets.interactive(full_widget_funct_all_data,
                            {"manual" : True},
                            username=text_user,
                            password=text_pass,
                            days_of_data=int_slider_days,
                            end_date=date_picker,
                           display_style=style_dropdown) 
    
    display(o)




display(out)
box = widgets.VBox([text_user, text_pass, date_picker, int_slider_days, style_dropdown])


widgets.interact(display_out)